In [1]:
import pandas as pd
import numpy as np
from IPython.display import display_markdown

import warnings
warnings.filterwarnings('ignore')

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Расчет сведений для МОНИТОРИНГА по СДВ, НС и ТРАНСМУК, по сведениям ЦЛО  <br>
</b>            </div>

### 
<div class="alert alert-block alert-warning">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Функции для расчета  <br>
</b>            </div>

#### функция 605го

In [2]:
def load_data_605(file_name, bmchu, bk, date_finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла 605го ----------------------
    
    # file_name = file_605
    df = pd.read_csv(file_name, delimiter=';', encoding='cp1251')

    # уберем лишние колонки
    df_605 = df.copy()
    df_605 = df_605[['Дата постановки на учет/включения пациента в реестр',
           'Пациент', 
           'Дата рождения', 'Пол',
           'Дата признания пациента нуждающимся в ПМП (ВК)',
           'Текущий статус пациента',
           'Дата прекращения оказания ПМП', 
             'Курирующее подразделение/филиал',
                   'Гражданский статус' ,
                    'Адрес', 'Основной диагноз по МКБ-10',
                     'Адрес фактический',
                     'Дата разрешения ДЗМ',
                     'Номер разрешения ДЗМ',
                     'БК'
                    ]]

    #  переименуем покороче столбцы
    df_605.rename(columns={'Дата постановки на учет/включения пациента в реестр': 'Дата_вкл_реестр'}, inplace=True)
    df_605.rename(columns={'Дата рождения': 'ДР'}, inplace=True)
    df_605.rename(columns={'Дата признания пациента нуждающимся в ПМП (ВК)': 'Дата_признания'}, inplace=True)
    df_605.rename(columns={'Дата прекращения оказания ПМП': 'Дата_прекр'}, inplace=True)
    df_605.rename(columns={'Курирующее подразделение/филиал': 'Филиал'}, inplace=True)
    df_605.rename(columns={'Основной диагноз по МКБ-10': 'Диагноз'}, inplace=True)
    df_605.rename(columns={'Гражданский статус': 'Иногород'}, inplace=True)

    df_concat = pd.concat([df_605])


    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр'] = pd.to_datetime(df_concat['Дата_вкл_реестр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()


    # Уберем записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'
    # - удаляем строки. в которых текущий статус не заполнен =  nan
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isin(['Новый пациент', 'Умер без п/о', 'Вне реестра', 'Врач КЦ']))]   # -----------------записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'----------------
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isna())]  # - удаляем строки. в которых текущий статус не заполнен =  nan


    # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bmchu != 'учитываем':
        df_concat = df_concat.loc[df_concat['Филиал'] != 'БМЧУ "Детский хоспис"']  # - удаляем строки. в которых текущий статус не заполнен =  nan
        
        
   #     # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bk != 'с учетом БК':
        df_concat = df_concat.loc[df_concat['БК'] != 'Да']  # - удаляем строки. в которых текущий статус не заполнен =  nan     

    # УДАЛИМ полные дубликаиы строк (оставляем только ПЕРВЫЕ строки из повторов (из 605го) )
    df_concat = df_concat.drop_duplicates(keep='first')
    df_concat.index.duplicated()
    df_concat.reset_index(drop= True , inplace= True )


    # Заполним поле Дата_первых_сведений данными: 
    #  если Дата_вкл_реестр не пустая, то  Дата_учета = Дата_вкл_реестр

    df_concat.loc[ (~df_concat['Дата_вкл_реестр'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_вкл_реестр']
    df_concat.loc[ (df_concat['Дата_вкл_реестр'].isnull()) & (~df_concat['Дата_признания'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_признания']
    df_concat.loc[ (df_concat['Дата_вкл_реестр'].isnull()) & (df_concat['Дата_признания'].isnull()) & (~df_concat['Дата_прекр'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_прекр'] - pd.Timedelta(days = 2)



    # новый признак - Дата_выбытия

    # 1 -- условие ---------------------------
    df_concat.loc[ (  
               ((df_concat['Дата_прекр'].isnull()) & (df_concat['Дата_вкл_реестр'] < '2020-12-31'))  
            &  (  (df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП') )
               )
               , 'Дата_выбытия'] = '01.01.1900'

    # 2 -- условие----------------------------
    df_concat.loc[(~df_concat['Дата_прекр'].isnull()), 'Дата_выбытия'] = df_concat['Дата_прекр']

    # 3 -- условие ---------------------------
    df_concat.loc[
                     (~df_concat['Дата_первых_сведений'].isnull() & df_concat['Дата_выбытия'].isnull() )  
                  &  ((df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП')), 'Дата_выбытия'] = df_concat['Дата_первых_сведений'] + pd.Timedelta(days = 2)



    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр'] = pd.to_datetime(df_concat['Дата_вкл_реестр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_выбытия'] = pd.to_datetime(df_concat['Дата_выбытия'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_первых_сведений'] = pd.to_datetime(df_concat['Дата_первых_сведений'], dayfirst=True, errors = 'coerce').dt.normalize()


    # ------------подсчитаем ВОЗРАСТ --------------

    
   # now = pd.Timestamp('now')
    now = pd.Timestamp(date_finish)
    df_concat.loc[(df_concat['Текущий статус пациента'] == 'Умер') & (~df_concat['Дата_выбытия'].isnull()), 'Возраст'] = (df_concat['Дата_выбытия'] - df_concat['ДР']).dt.days//365
    df_concat.loc[(df_concat['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (now - df_concat['ДР']).astype('<m8[Y]')




    #  ------ посчитаем категорию трудоспособный --------
    #  если пол ЖЕНСКИЙ, и возраст Менее 57 -----> Категория - трудоспособный
    #  если пол Мужской, и возраст Менее 62 -----> Категория - трудоспособный

    df_concat['Категория'] = np.nan
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Мужской') & (df_concat['Возраст'] >= 62), 'Категория'] = 'Старше трудосп'
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Женский') & (df_concat['Возраст'] >= 57), 'Категория'] = 'Старше трудосп'


    #  ------ определим детей ---------------
    df_concat['Дети'] = np.nan
    df_concat.loc[(df_concat['Возраст'] < 18) & (df_concat['Возраст'] >= 0), 'Дети'] = 'Дети'



    #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------

    df_concat['Ключ'] = df_concat['Пациент'].map(str) + (df_concat['ДР'].dt.date).map(str)
    df_concat['Ключ'] = df_concat['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_concat['Ключ'] = df_concat['Ключ'].str.replace(' ', '')   # ---- пробелы долой 


    df_clr = df_concat.loc[ (~df_concat['Дата_первых_сведений'].isnull())]

    # ------ удаление дубликатов ----------
    df_clr = df_clr.drop_duplicates(['Ключ'], keep='first') 
    
    return df_clr  

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция формирования датафрейма ПРЕПАРАТА (универс)  <br></b>            </div>

In [3]:
def load_preparat(file_name, date_finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла preparat ----------------------
    df_preparat = pd.read_csv(file_name, delimiter=';', encoding='cp1251')


        # ----------------------- Уберем строки, в которых нет  даных о выдаче препаратов --------  (препараты должны быть с 5й строки)------    
    df_preparat = df_preparat[df_preparat[df_preparat.columns[5:]].notna().any(axis=1)]
    
    
    # уберем лишние колонки
    df_preparat = df_preparat[['№', 'Серия и номер полиса ОМС', 'СНИЛС', 'ФИО пациента',   'Дата рождения']]
    
    

    # ---- преобразуем дату в тип дата -----------
    df_preparat['Дата рождения'] = pd.to_datetime(df_preparat['Дата рождения'], dayfirst=True, errors = 'coerce').dt.normalize()

    # ------ Добавим поле ВОЗРАСТ и ДЕТИ------------
    df_preparat['Возраст'] = np.nan
    df_preparat['Дети'] = np.nan
    # ------------подсчитаем ВОЗРАСТ --------------
  # now = pd.Timestamp('now')
    now = pd.Timestamp(date_finish)
    df_preparat['Возраст'] = (now - df_preparat['Дата рождения']).astype('<m8[Y]')
    df_preparat.loc[df_preparat['Возраст'] < 18,  'Дети'] = 'Дети'

    #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------
    df_preparat['Ключ'] = df_preparat['ФИО пациента'].map(str) + (df_preparat['Дата рождения'].dt.date).map(str)
    df_preparat['Ключ'] = df_preparat['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_preparat['Ключ'] = df_preparat['Ключ'].str.replace(' ', '')   # ---- пробелы долой 

    # ----- Заполним новый признак о всех пацментах датафрейма сведениями о полученных препаратах (т.к. все они из файла ЦЛО о получивших трансмук)
    df_preparat['preparat'] = 'preparat'
    
    return df_preparat

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета Количество пациентов, получателей помощи (чел.)  <br></b>            </div>

In [4]:
# 4444 ---------------------------------------------  Количество пациентов, получателей помощи (чел.)
def def_data_all_clients_data_vkl_reestr(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------

    df_all = df.loc[ ((df['Дата_первых_сведений'] < finish)  
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))]

    data_all_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)  
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))].shape[0] 


    data_all_clients_trud = df.loc[ ((df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)))  
                             &   (  df['Категория'] == 'Старше трудосп')        
                                  ].shape[0]
    data_all_clients_deti = df.loc[ ((df['Дата_первых_сведений'] < finish) 
                     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)))  
                     # &   (  df['Категория'] == 'Старше трудосп')          
                    &   (  ~df['Дети'].isna())          
                                  
                                  ].shape[0]
    
    
    return  df_all, data_all_clients, data_all_clients_trud, data_all_clients_deti, data_all_clients 

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция Получатели (универсальная) <br></b>            </div>

In [5]:
def poluchateli_v_reestre(name_605, name_preparat, start, finish, bmchu, bk):
    
    # ---------------- формируем df по чистому 605-му ----
    df_cpp = load_data_605(name_605, bmchu, bk, finish)
    # ---------------- формируем df по НС ----
    df_preparat = load_preparat(name_preparat, finish)    
        
    # ----------- функция подсчета Количество пациентов, получателей помощи (чел.) --
    df_605_period = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[0]
    df_605_period_trud = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[2]
    
#     # ----------- Оставим только нужные поля 605го для соединения с данными СДВ -------------
    df_605_period = df_605_period[['Пациент', 'ДР', 'Ключ', 'Категория']]

    df_mer = df_605_period.merge(df_preparat[['Ключ','preparat', 'Возраст', 'Дети']], how='left', on='Ключ')
# 
#     # ----------- Взрослые preparat (уникальные) -------------
    df_preparat_vzros = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] > 17)].drop_duplicates()
#     # ----------- Взрослые preparat СТАРШЕ ТРУД (уникальные) -------------
    df_preparat_vzros_trud = df_preparat_vzros.loc[~df_preparat_vzros.Категория.isna()].drop_duplicates()
#     # ----------- Взрослые Дети (уникальные) -------------
    df_preparat_deti = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] <= 17)].drop_duplicates()

    

    return df_605_period.shape[0], df_605_period_trud, df_preparat_vzros.shape[0], df_preparat_vzros_trud.shape[0], df_preparat_deti.shape[0], def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[3], df_preparat_deti
    

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета Количество пациентов, получателей помощи (чел.)  <br></b>            </div>

In [6]:
# 4444 ---------------------------------------------  Количество пациентов, получателей помощи (чел.)
def def_data_all_clients(df, start, finish):
    
 # ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------

    df_all = df.loc[ ((df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))

                    ]

    data_all_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)   &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))

                             ]

    data_all_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                             &   (  df['Дети'].isna())
                                    
                                   ]
    
    data_all_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                             &   (  df['Категория'] == 'Старше трудосп')    

                                        ]
    
    data_all_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish) 
                         &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                         &   (  ~df['Дети'].isna()) 

                                  ]
    
    return data_all_clients.shape[0],  data_all_clients_vzros.shape[0], data_all_clients_vzros_trud.shape[0], data_all_clients_deti.shape[0], data_all_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета НОВЫХ пациентов, обратившихся в ЭТОМ периоде (чел.)  <br></b>            </div>

In [7]:
# 4444 ---------------------------------------------  Количество НОВЫХ пациентов, получателей помощи (чел.)
def def_data_new_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------Дата_первых_сведений


    data_new_clients = df.loc[ ((df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish) )

                             ]


    data_new_clients_vzros = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish)   
                             &   (  df['Дети'].isna())
                                    
                                   ]
    
    data_new_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish)   
                             &   (  df['Категория'] == 'Старше трудосп') 

                                        ]
    
    data_new_clients_deti = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish) 
                         &   (  ~df['Дети'].isna())    

                                  ]
    
    return data_new_clients.shape[0],  data_new_clients_vzros.shape[0], data_new_clients_vzros_trud.shape[0], data_new_clients_deti.shape[0], data_new_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета УМЕРШИХ пациентов, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [8]:
# 4444 ---------------------------------------------  Количество НОВЫХ пациентов, получателей помощи (чел.)--------------------------------------Дата_первых_сведений
def def_data_dead_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------


    data_dead_clients = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                              & (df['Текущий статус пациента'] == 'Умер')
                              ]


    data_dead_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                                   &   (  df['Дети'].isna())
                                    & (df['Текущий статус пациента'] == 'Умер')
                                   ]
    
    data_dead_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  ) 
                                 &   (  df['Категория'] == 'Старше трудосп') 
                                  & (df['Текущий статус пациента'] == 'Умер')
                                        ]
    
    data_dead_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish)      &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                                   & (  ~df['Дети'].isna())                                     
                                   & (df['Текущий статус пациента'] == 'Умер')
                                  ]
    
    return data_dead_clients.shape[0],  data_dead_clients_vzros.shape[0], data_dead_clients_vzros_trud.shape[0], data_dead_clients_deti.shape[0], data_dead_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета ИНОГОРОДНИХ пациентов, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [9]:
# 100723 ----------------------------------------------ИНОГОРОДНИЕ-----------------------
def def_data_inogorod_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------  'Гражданин РФ/иногородний', 'Украина', 'БОМЖ', nan, 'Иностранец', 'ЛНР', 'ДНР'


    data_inogorod_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))
                             
                                   & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   ) ]


    
    data_inogorod_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )  
                                   &   (  df['Дети'].isna())
                                   & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    data_inogorod_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )   
                                 &   (  df['Категория'] == 'Старше трудосп') 
                                  & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    data_inogorod_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish)      &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ) 
                                   & (  ~df['Дети'].isna())                                     
                                  & (
                                      (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний'])) 
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    return data_inogorod_clients.shape[0],  data_inogorod_clients_vzros.shape[0], data_inogorod_clients_vzros_trud.shape[0], data_inogorod_clients_deti.shape[0], data_inogorod_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета ЖИВЫХ пациентов на СЕГОДНЯ (ел.)  <br></b>            </div>

In [37]:
# 100723 ---------------------------------------------- ЖИВЫХ пациентов на СЕГОДНЯ-----------------------
def def_live_clients_today(df):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------  'Гражданин РФ/иногородний', 'Украина', 'БОМЖ', nan, 'Иностранец', 'ЛНР', 'ДНР'


    live_all_clients_today = df.loc[ (df['Текущий статус пациента'].isin(['Патронаж', 'Госпитализация в ОПМП', 'Стационар ЦПП']))]
                                    
    live_vzros_clients_today = df.loc[ (df['Текущий статус пациента'].isin(['Патронаж', 'Госпитализация в ОПМП', 'Стационар ЦПП']))
                                    & (  df['Дети'].isna())
                                    ]
                                    
    live_deti_clients_today = df.loc[ (df['Текущий статус пациента'].isin(['Патронаж', 'Госпитализация в ОПМП', 'Стационар ЦПП']))
                                   & ( ~df['Дети'].isna())
                                    ]
                             
    
    
    return live_all_clients_today.shape[0],  live_vzros_clients_today.shape[0], live_deti_clients_today.shape[0], live_deti_clients_today

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=6>
<b> РЕЗУЛЬТАТ Статистика МОНИТОРИНГ <br></b>            </div>

In [45]:
# ---------------------------- ИТОГИ ---------------
start = '2022-01-01'
finish = '2023-08-03'
# file_605 = '605_0404.csv'
# file_605 = '605_рц_бк_1204.csv'
file_605 = '605_0308_дети.csv'

# bmchu = 'не учитываем'
bmchu = 'учитываем'
bk = 'с учетом БК'
# bk = 'без учета БК'


god_start = start.split('-')[0]
mes_start = start.split('-')[1]
day_start = start.split('-')[2]

god_finish = finish.split('-')[0]
mes_finish = finish.split('-')[1]
day_finish = finish.split('-')[2]


# ------------------------------------------------готовим реестр ------
df_reest = load_data_605(file_605, bmchu, bk,finish)

# ------------------------------------------------собираем данные ------
all_clients  = def_data_all_clients(df_reest, start, finish)
new_clients  = def_data_new_clients(df_reest, start, finish)
dead_clients = def_data_dead_clients(df_reest, start, finish)
inogorod_clients = def_data_inogorod_clients(df_reest, start, finish)
# data_inogorod_clients_all = def_data_inogorod_clients_all(df_reest, start, finish)

df_live = def_live_clients_today(df_reest)

a = f'Статистика МОНИТОРИНГ период с {day_start}.{mes_start}.{god_start} по {day_finish}.{mes_finish}.{god_finish}:'

display_markdown(f''' # {a}

| Параметр | Количество всего| Взрослые |трудоспособ | Дети |
|---:|:-|:-|:-|:-|
| Количество пациентов __ВСЕГО__ за период __БМЧУ__{bmchu}__ и __{bk}__| {all_clients[0]}      | {all_clients[1]} |{all_clients[2]} |{all_clients[3]} |
| Количество __НОВЫХ__ пациентов за период          | {new_clients[0]}      | {new_clients[1]} |{new_clients[2]} |{new_clients[3]} |
| Количество __УМЕРШИХ__ пациентов за период        | {dead_clients[0]}     | {dead_clients[1]} |{dead_clients[2]} |{dead_clients[3]} |
| Количество __ИНОГОРОДНИХ__ пациентов за период    | {inogorod_clients[0]} | {inogorod_clients[1]} |{inogorod_clients[2]} |{inogorod_clients[3]} |
| в __ИНОГОРОДНИХ__ __{bk}__ учтены:                | ---- |---- |---- |---- |
| __'Гражданин РФ/иногородний', 'ЛНР', 'ДНР'__      | ---- |---- |---- |---- |


''', raw=True)

 # Статистика МОНИТОРИНГ период с 01.01.2022 по 03.08.2023:

| Параметр | Количество всего| Взрослые |трудоспособ | Дети |
|---:|:-|:-|:-|:-|
| Количество пациентов __ВСЕГО__ за период __БМЧУ__учитываем__ и __с учетом БК__| 37427      | 36296 |30917 |1131 |
| Количество __НОВЫХ__ пациентов за период          | 27629      | 27140 |23319 |489 |
| Количество __УМЕРШИХ__ пациентов за период        | 23040     | 22838 |19691 |202 |
| Количество __ИНОГОРОДНИХ__ пациентов за период    | 312 | 276 |171 |36 |
| в __ИНОГОРОДНИХ__ __с учетом БК__ учтены:                | ---- |---- |---- |---- |
| __'Гражданин РФ/иногородний', 'ЛНР', 'ДНР'__      | ---- |---- |---- |---- |




In [26]:
all_clients[4].to_excel('реестр_2022.xlsx')

In [46]:
df_live[3]

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
2119,NaT,Василенко Богдана Александровна,2005-10-19,Женский,2023-03-01,Стационар ЦПП,NaT,Координационный центр,ЛНР,NaN,...,NaN,NaN,NaN,Нет,2023-03-01,NaT,17.0,NaN,Дети,василенкобогданаалександровна2005-10-19
2120,2023-05-29,Ляшук Арина Валериевна,2023-03-03,Женский,2023-05-22,Госпитализация в ОПМП,NaT,ПМДХ ОВПП,ДНР,NaN,...,"141551, Московская обл, Солнечногорск, Андреев...",NaN,NaN,Нет,2023-05-29,NaT,0.0,NaN,Дети,ляшукаринавалериевна2023-03-03
2121,2023-07-28,Атаян Милана Анушавановна,2023-06-25,Женский,2023-07-25,Патронаж,NaT,ПМДХ ОВПП,Москвич,"Москва г, вн.тер.г. поселение Внуковское, Омск...",...,NaN,NaN,NaN,Нет,2023-07-28,NaT,0.0,NaN,Дети,атаянмиланаанушавановна2023-06-25
2122,2023-07-19,Фатеева Анна Дмитриевна,2023-04-17,Женский,2023-07-04,Стационар ЦПП,NaT,ПМДХ ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Кунцев...",...,NaN,NaN,NaN,Нет,2023-07-19,NaT,0.0,NaN,Дети,фатеевааннадмитриевна2023-04-17
2123,2023-06-26,Цветков Григорий Павлович,2023-02-18,Мужской,2023-07-05,Госпитализация в ОПМП,NaT,"БМЧУ ""Детский хоспис""",Москвич,"Москва г, Солдатская ул, Дом 3, кв. 334",...,"Москва г, Солдатская ул, Дом 3, кв. 334",NaN,NaN,Нет,2023-06-26,NaT,0.0,NaN,Дети,цветковгригорийпавлович2023-02-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91593,2023-05-29,Зиновьев Платон Станиславович,2014-08-22,Мужской,NaT,Патронаж,NaT,"БМЧУ ""Детский хоспис""",Москвич,"Москва г, дом 44, пр-д грайвороновский 2-й, к....",...,"Москва г, дом 44, пр-д грайвороновский 2-й, к....",NaN,NaN,Нет,2023-05-29,NaT,8.0,NaN,Дети,зиновьевплатонстаниславович2014-08-22
92674,2023-06-19,Эльгукаева Аиша Сухрабовна,2020-06-26,Женский,NaT,Патронаж,NaT,"БМЧУ ""Детский хоспис""",Москвич,"Москва, ул. Профсоюзная, д. 64, к. 2, кв. 160",...,"Москва, ул. Профсоюзная, д. 64, к. 2, кв. 160",NaN,NaN,Нет,2023-06-19,NaT,3.0,NaN,Дети,эльгукаевааишасухрабовна2020-06-26
93872,2023-07-14,Попов Илья Антонович,2023-02-28,Мужской,NaT,Патронаж,NaT,"БМЧУ ""Детский хоспис""",Москвич,"Москва г, вн.тер.г. муниципальный округ Басман...",...,"Москва г, вн.тер.г. муниципальный округ Басман...",NaN,NaN,Нет,2023-07-14,NaT,0.0,NaN,Дети,поповильяантонович2023-02-28
94439,2023-07-24,Харин Роман Романович,2017-01-23,Мужской,NaT,Патронаж,NaT,"БМЧУ ""Детский хоспис""",Москвич,"Москва г, Москва, ул. Дмитрия Ульянова, д. 43,...",...,"Москва г, Москва, ул. Дмитрия Ульянова, д. 43,...",NaN,NaN,Нет,2023-07-24,NaT,6.0,NaN,Дети,харинроманроманович2017-01-23


In [29]:
# df_live[3].to_excel('реестр_дети_0308.xlsx')

In [47]:
df_live[3].loc[df_live[3]['Пациент'] == 'Кутепов Мирон Артемович']

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
2366,2022-05-26,Кутепов Мирон Артемович,2020-06-29,Мужской,2022-05-04,Стационар ЦПП,NaT,ПМДХ ОВПП,Москвич,"117041, Москва г, муниципальный округ Южное Бу...",...,"117042, Москва г, муниципальный округ Южное Бу...",NaN,NaN,Нет,2022-05-26,NaT,3.0,NaN,Дети,кутеповмиронартемович2020-06-29


In [48]:
df_live[3].loc[df_live[3]['Пациент'] == 'Ляшук Арина Валериевна']

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
2120,2023-05-29,Ляшук Арина Валериевна,2023-03-03,Женский,2023-05-22,Госпитализация в ОПМП,NaT,ПМДХ ОВПП,ДНР,NaN,...,"141551, Московская обл, Солнечногорск, Андреев...",NaN,NaN,Нет,2023-05-29,NaT,0.0,NaN,Дети,ляшукаринавалериевна2023-03-03


In [41]:
df_reest.loc[df_reest['Пациент'] == 'Ляшук Арина Валериевна']

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
2120,2023-05-29,Ляшук Арина Валериевна,2023-03-03,Женский,2023-05-22,Госпитализация в ОПМП,NaT,ПМДХ ОВПП,ДНР,NaN,...,"141551, Московская обл, Солнечногорск, Андреев...",NaN,NaN,Нет,2023-05-29,NaT,-1.0,NaN,NaN,ляшукаринавалериевна2023-03-03


In [36]:
df_reest.loc[df_reest['Пациент'] == 'Кутепов Мирон Артемович']

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
2366,2022-05-26,Кутепов Мирон Артемович,2020-06-29,Мужской,2022-05-04,Стационар ЦПП,NaT,ПМДХ ОВПП,Москвич,"117041, Москва г, муниципальный округ Южное Бу...",...,"117042, Москва г, муниципальный округ Южное Бу...",NaN,NaN,Нет,2022-05-26,NaT,2.0,NaN,Дети,кутеповмиронартемович2020-06-29


In [43]:
#  ************
#  ------------------------ ПОЧЕМУ ВОЗРАСТ отрицательный??? -------------------

In [54]:
df_reest.loc[df_reest['Возраст'] < 0]

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
2055,2021-04-27,Радина Неонила Ивановна,1929-09-26,Женский,2021-05-29,Умер,1929-06-30,"Центр ОВПП(ул. Двинцев, 6, стр. 2, Москва, Рос...",Москвич,"127238, Москва г, вн.тер.г. муниципальный окру...",...,"Москва г, Дмитровское ш, Дом 42",NaN,NaN,Нет,2021-04-27,1929-06-30,-1.0,NaN,NaN,радинанеонилаивановна1929-09-26
5649,NaT,Соловьева Александра Павловна,2008-01-24,Женский,NaT,Умер,2000-03-30,"Центр ОВПП(ул. Двинцев, 6, стр. 2, Москва, Рос...",NaN,"г.Москва, Южнобутовская улица, д.19, кв.дом",...,"Москва г, Академика Павлова ул, Дом 15, Строен...",NaN,NaN,Нет,2000-03-28,2000-03-30,-8.0,NaN,NaN,соловьеваалександрапавловна2008-01-24
6496,2010-01-25,Иванов Александр Николаевич,1956-07-09,Мужской,NaT,Умер,NaT,Бутово ОВПП,NaN,NaN,...,"Пос.Липки ,Коммунистическая,6",NaN,NaN,Нет,2010-01-25,1900-01-01,-57.0,NaN,NaN,ивановалександрниколаевич1956-07-09
6503,2010-07-12,БИТКОВ АНАТОЛИЙ ФЕДОРОВИЧ,1942-09-18,Мужской,NaT,Умер,NaT,Бутово ОВПП,NaN,NaN,...,ОСТРОВИТЯНОВА 53-272,NaN,NaN,Нет,2010-07-12,1900-01-01,-43.0,NaN,NaN,биткованатолийфедорович1942-09-18
6504,2010-07-12,ГРОМОВ АЛЕКСЕЙ ФЕДОРОВИЧ,1926-02-01,Мужской,NaT,Умер,NaT,Бутово ОВПП,NaN,NaN,...,NaN,NaN,NaN,Нет,2010-07-12,1900-01-01,-27.0,NaN,NaN,громовалексейфедорович1926-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55819,2020-12-07,Поленова Надежда Васильевна,1939-07-13,Женский,NaT,Умер,NaT,"Центр ОВПП(ул. Двинцев, 6, стр. 2, Москва, Рос...",NaN,NaN,...,NaN,NaN,NaN,Нет,2020-12-07,1900-01-01,-40.0,NaN,NaN,поленованадеждавасильевна1939-07-13
56454,2020-12-25,Абрамов Сергей Сергеевич,1923-02-05,Мужской,NaT,Умер,NaT,Царицыно ОВПП,Москвич,NaN,...,Бакинская 4-121,NaN,NaN,Нет,2020-12-25,1900-01-01,-24.0,NaN,NaN,абрамовсергейсергеевич1923-02-05
56470,2020-12-25,Дудченко Людмила Николаевна,1930-06-04,Женский,NaT,Умер,NaT,"Центр ОВПП(ул. Двинцев, 6, стр. 2, Москва, Рос...",NaN,NaN,...,NaN,NaN,NaN,Нет,2020-12-25,1900-01-01,-31.0,NaN,NaN,дудченколюдмиланиколаевна1930-06-04
63197,2021-06-24,Климова Галина Ивановна,1944-09-28,Женский,NaT,Умер,1944-09-01,Перово ОВПП,Москвич,"Гончарная 26-1-7, эт.6, п.1, код: 007",...,"Гончарная 26-1-7, эт.6, п.1, код: 007",NaN,NaN,Нет,2021-06-24,1944-09-01,-1.0,NaN,NaN,климовагалинаивановна1944-09-28


In [71]:
now = pd.Timestamp('now')
# now = pd.Timestamp(date_finish)
df_reest.loc[(df_reest['Текущий статус пациента'] == 'Умер') & (~df_reest['Дата_выбытия'].isnull()), 'Возраст'] = (df_reest['Дата_выбытия'] - df_reest['ДР']).dt.days//365
df_reest.loc[(df_reest['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (now - df_reest['ДР']).astype('<m8[Y]')
df_reest

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
0,2023-02-02,Зинатшина Анастасия Владимировна,1986-09-22,Женский,2023-01-31,Умер,2023-02-03,Зеленоград ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Савелк...",...,"124482, Москва г, вн.тер.г. муниципальный окру...",NaN,NaN,Нет,2023-02-02,2023-02-03,36.0,NaN,NaN,зинатшинаанастасиявладимировна1986-09-22
1,2023-06-02,Олейникова Александра Павловна,1985-03-04,Женский,NaT,Умер,2023-06-28,Зеленоград ОВПП,Москвич,"109341, Москва г, муниципальный округ Марьино,...",...,"141551, Московская обл, Солнечногорск, Голубое...",NaN,NaN,Нет,2023-06-02,2023-06-28,38.0,NaN,NaN,олейниковаалександрапавловна1985-03-04
2,2020-05-15,Волченкова Юлия Викторовна,1983-05-22,Женский,2022-03-09,Прекращено оказание ПМП,2023-01-17,Зеленоград ОВПП,Москвич,"Москва г, Зеленоград г, корпус 810, квартира 110",...,"124527, Москва г, Зеленоград г, корпус 810, кв...",NaN,NaN,Нет,2020-05-15,2023-01-17,40.0,NaN,NaN,волченковаюлиявикторовна1983-05-22
3,2020-04-23,Лихачева Татьяна Михайловна,1979-11-10,Женский,NaT,Умер,2020-06-21,Зеленоград ОВПП,Москвич,"г.Москва, Сиреневый бульвар, д.56, кв.112",...,"124617, Москва г, вн.тер.г. муниципальный окру...",NaN,NaN,Нет,2020-04-23,2020-06-21,40.0,NaN,NaN,лихачевататьянамихайловна1979-11-10
4,2023-03-03,Комарова Анастасия Анатольевна,1978-01-17,Женский,2023-03-01,Умер,2023-03-05,Зеленоград ОВПП,Москвич,"124575, Москва г, Зеленоград г, корпус 930, кв...",...,"124536, Москва г, Зеленоград г, корпус 511, кв...",NaN,NaN,Нет,2023-03-03,2023-03-05,45.0,NaN,NaN,комароваанастасияанатольевна1978-01-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94926,2033-02-08,Бородина Валентина Вениаминовна,1945-01-30,Женский,2023-01-27,Стационар ЦПП,NaT,Люблино ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Марьин...",...,"Москва г, вн.тер.г. муниципальный округ Марьин...",NaN,NaN,Нет,2033-02-08,NaT,78.0,Старше трудосп,NaN,бородинавалентинавениаминовна1945-01-30
94927,2099-08-07,Муллаянов Рафаиль Мухлисуллович,1954-11-13,Мужской,2022-09-07,Умер,2022-10-17,Царицыно ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Орехов...",...,"115563, Москва г, вн.тер.г. муниципальный окру...",NaN,NaN,Нет,2099-08-07,2022-10-17,67.0,Старше трудосп,NaN,муллаяноврафаильмухлисуллович1954-11-13
94928,2202-04-12,Еникова Феня Мартковна,1925-11-18,Женский,NaT,Умер,2021-04-20,ПМХ ОВПП,Гражданин РФ/иногородний,Крым Феодосия Украинская 11,...,"Москва Западный Раменки Удальцова 89 кв 52, п...",NaN,NaN,Нет,2202-04-12,2021-04-20,95.0,Старше трудосп,NaN,ениковафенямартковна1925-11-18
94929,2221-05-02,Тимофеева Татьяна Константиновна,1936-06-20,Женский,2021-05-14,Умер,2021-05-17,Хорошево ОВПП,Москвич,Москва Западный Фили-Давыдково Большая Филёвск...,...,Москва Западный Фили-Давыдково Большая Филёвск...,NaN,NaN,Нет,2221-05-02,2021-05-17,84.0,Старше трудосп,NaN,тимофеевататьянаконстантиновна1936-06-20


In [72]:
df_reest.loc[
    (df_reest['Дата_прекр'].isna())
    &
    (df_reest['ДР'] >= '2005-01-01') &  (df_reest['ДР'] <= '2005-12-31') ] 

,Дата_вкл_реестр,Пациент,ДР,Пол,Дата_признания,Текущий статус пациента,Дата_прекр,Филиал,Иногород,Адрес,...,Адрес фактический,Дата разрешения ДЗМ,Номер разрешения ДЗМ,БК,Дата_первых_сведений,Дата_выбытия,Возраст,Категория,Дети,Ключ
2119,NaT,Василенко Богдана Александровна,2005-10-19,Женский,2023-03-01,Стационар ЦПП,NaT,Координационный центр,ЛНР,NaN,...,NaN,NaN,NaN,Нет,2023-03-01,NaT,17.0,NaN,Дети,василенкобогданаалександровна2005-10-19
2259,2023-07-29,Антонова Анна Олеговна,2005-09-11,Женский,2023-07-20,Патронаж,NaT,ПМДХ ОВПП,Москвич,"108815, Москва г, Филимонковское п, Братьев Гр...",...,NaN,NaN,NaN,Нет,2023-07-29,NaT,17.0,NaN,Дети,антоновааннаолеговна2005-09-11
2260,2023-04-05,Колупаев Константин Андреевич,2005-03-08,Мужской,2023-03-29,Патронаж,NaT,Хорошево ОВПП,Москвич,"124365, Москва г, Зеленоград г, Георгиевский п...",...,"Москва г, вн.тер.г. муниципальный округ Хороше...",NaN,NaN,Нет,2023-04-05,NaT,18.0,NaN,NaN,колупаевконстантинандреевич2005-03-08
2261,2023-03-04,Панкратов Герман Савельевич,2005-05-24,Мужской,2023-02-20,Патронаж,NaT,Преображенское,Москвич,"107258, Москва г, муниципальный округ Богородс...",...,"107258, Москва г, муниципальный округ Богородс...",NaN,NaN,Нет,2023-03-04,NaT,18.0,NaN,NaN,панкратовгермансавельевич2005-05-24
2262,2023-03-13,Романенко Никита Сергеевич,2005-07-21,Мужской,2023-03-01,Госпитализация в ОПМП,NaT,"БМЧУ ""Детский хоспис""",Москвич,"108830, Москва г, поселение Вороновское, ЛМС п...",...,"108830, Москва г, поселение Вороновское, ЛМС п...",NaN,NaN,Нет,2023-03-13,NaT,18.0,NaN,NaN,романенконикитасергеевич2005-07-21
2263,2023-02-06,Петров Данил Алексеевич,2005-07-13,Мужской,2023-01-25,Патронаж,NaT,ПМДХ ОВПП,Москвич,"127410, Москва г, вн.тер.г. муниципальный окру...",...,"127410, Москва г, вн.тер.г. муниципальный окру...",NaN,NaN,Нет,2023-02-06,NaT,18.0,NaN,NaN,петровданилалексеевич2005-07-13
2485,2023-02-03,Макарова Дарья Дмитриевна,2005-05-10,Женский,2019-10-10,Патронаж,NaT,"БМЧУ ""Детский хоспис""",Москвич,"Москва г, Зеленоград г, корпус 348, квартира 60",...,NaN,NaN,NaN,Нет,2023-02-03,NaT,18.0,NaN,NaN,макаровадарьядмитриевна2005-05-10
2487,2022-07-04,Силичева Анастасия Васильевна,2005-06-05,Женский,2022-06-24,Патронаж,NaT,Царицыно ОВПП,Москвич,"Москва г, вн.тер.г. муниципальный округ Бирюле...",...,"Москва г, вн.тер.г. муниципальный округ Бирюле...",NaN,NaN,Нет,2022-07-04,NaT,18.0,NaN,NaN,силичеваанастасиявасильевна2005-06-05
2488,2022-06-03,Гурьев Кирилл Алексеевич,2005-03-03,Мужской,2022-05-30,Патронаж,NaT,Вороново ОВПП,Москвич,"121471, Москва г, дом 8, п.Вороновское д.семён...",...,"121471, Москва г, дом 8, п.вороновское д.семён...",NaN,NaN,Нет,2022-06-03,NaT,18.0,NaN,NaN,гурьевкириллалексеевич2005-03-03
2489,2022-05-11,Лобанов Дмитрий Евгеньевич,2005-04-27,Мужской,2022-04-13,Патронаж,NaT,Некрасовка ОВПП,Москвич,"111672, Москва г, муниципальный округ Новокоси...",...,"111672, Москва г, муниципальный округ Новокоси...",NaN,NaN,Нет,2022-05-11,NaT,18.0,NaN,NaN,лобановдмитрийевгеньевич2005-04-27


In [ ]:
# df_dead_1204 = dead_clients[4]
# df_dead_1204['period'] = 'apr'
# df_dead_1204
df_dead_0407 = dead_clients[4]
df_dead_0407['period'] = 'jul'
df_dead_0407

In [ ]:
df_dead_1204

In [ ]:
 # ------------подсчитаем ВОЗРАСТ --------------

    
   # now = pd.Timestamp('now')
    now = pd.Timestamp(date_finish)
    df_concat.loc[(df_concat['Текущий статус пациента'] == 'Умер') & (~df_concat['Дата_выбытия'].isnull()), 'Возраст'] = (df_concat['Дата_выбытия'] - df_concat['ДР']).dt.days//365
    df_concat.loc[(df_concat['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (now - df_concat['ДР']).astype('<m8[Y]')

In [ ]:
df_comp = pd.merge(df_dead_0407, df_dead_1204, how='left', on='Ключ')
df_comp

In [ ]:
df_only_new = df_comp.loc[df_comp['period_y'].isna()]
df_only_new

In [ ]:
df_dead_1204.loc[df_dead_1204['Пациент'] =='Захаров Александр Александрович']

In [ ]:
df_dead_0407.loc[df_dead_0407['Пациент'] =='Захаров Александр Александрович']

In [ ]:
df_only_new.loc[df_only_new['period_x'] =='jul']

In [ ]:
df_dead_1204

### 
<div class="alert alert-block alert-warning">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Функции для расчета ПОСЕЩЕНИЙ <br>
</b>            </div>

#### функция ВПС

In [ ]:
def load_data_vps(file_name, date_finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла VPS ----------------------

    df_vps = pd.read_csv(file_name, delimiter=';', encoding='cp1251')

    # уберем лишние колонки
    
    df_vps = df_vps[['Взаимодействие.Вид взаимодействия',
           'Взаимодействие.Клиент.Наименование', 
                     # 'Взаимодействие.Ответственный',
           'Взаимодействие.Подразделение', 
                     'Клиент.БК', 
                     'Клиент.Наблюдение в РЦ',
           # 'Клиент.Адрес фактический',
                     'Клиент.Дата рождения',
           # 'Дата завершения взаимодействия', 
                     'Клиент.Пол',
           'Взаимодействие.Плановая дата', 
                     'Выдано_респ_оборуд'
                     # 'Количество записей'
                     
                    ]]
        #  переименуем покороче столбцы
    df_vps.rename(columns={'Взаимодействие.Вид взаимодействия': 'Вид_визита'}, inplace=True)
    df_vps.rename(columns={'Взаимодействие.Клиент.Наименование': 'Пациент'}, inplace=True)
    df_vps.rename(columns={'Взаимодействие.Подразделение': 'Филиал'}, inplace=True)
    df_vps.rename(columns={'Клиент.Наблюдение в РЦ': 'Набл_РЦ'}, inplace=True)
    df_vps.rename(columns={'Клиент.Дата рождения': 'ДР'}, inplace=True)
    df_vps.rename(columns={'Клиент.Пол': 'Пол'}, inplace=True)
    df_vps.rename(columns={'Взаимодействие.Плановая дата': 'Дата_визита'}, inplace=True)
    df_vps.shape[0]   
        
                # --------------------старый  набор ВИЗИТОВ--------------------
    df_vps = df_vps.loc[(~df_vps['Вид_визита'].isin([
     #      'Первичный визит врача КЦ',
     #     'Первичный визит врача ОВПП', 
     # 'Первичный визит заведующего ОВПП',
    #     'Визит врача КЦ', 
        'Звонок - контроль состояния', 
        'Звонок врача - контроль состояния',
        'Звонок медицинской сестры - контроль состояния',
             'Звонок врача-подтверждение визита', 'Телефонный звонок', 'Итого'  
               'Звонок - консультация социального работника',            
           'Звонок-подтверждение визита', 
        'Визит младшей медицинской сестры',
           # ' Визит медбрата',
        'Визит социального работника', 'Итого',
        'Звонок - консультация социального работника',
             'Звонок - контроль получения НС',
        'Звонок врача - контроль состояния НЕ ИСПОЛЬЗОВАТЬ',
    ]))] 

    

    df_vps = df_vps.drop_duplicates(keep='first')


    
        # ----------------форматнем дату к дате
    df_vps['ДР'] = pd.to_datetime(df_vps['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_vps['Дата_визита'] = pd.to_datetime(df_vps['Дата_визита'], dayfirst=True, errors = 'coerce').dt.normalize()
    
       # ------------подсчитаем ВОЗРАСТ --------------на дату ФИНИША!!!

    # now = pd.Timestamp('now')
    now = pd.Timestamp(date_finish)
    df_vps.loc[ (~df_vps['Дата_визита'].isnull()), 'Возраст'] = (df_vps['Дата_визита'] - df_vps['ДР']).dt.days//365

    
        #  ------ определим детей ---------------
    df_vps['Дети'] = np.nan
    df_vps.loc[(df_vps['Возраст'] < 18) & (df_vps['Возраст'] >= 0), 'Дети'] = 'Дети'
    
    
    
        #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------

    df_vps['Ключ'] = df_vps['Пациент'].map(str) + (df_vps['ДР'].dt.date).map(str)
    df_vps['Ключ'] = df_vps['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_vps['Ключ'] = df_vps['Ключ'].str.replace(' ', '')   # ---- пробелы долой 
    

    return df_vps



### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция формирования Датафрейма данных по выездам за период  <br></b>            </div>

In [ ]:
def f_create_df_vizit_vps(file_name, start, finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла VPS ----------------------

    df = load_data_vps(file_name, finish)
    
    df = df.loc[ (df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish )]
    
    #  ------------------------------------------Взрослые ВСЕГО -------
    df_vzrosl_all = df.loc[ 
                           (df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish )
                        &  ((df['Дети'].isna()))
                                &  (df['Клиент.БК'] != 'Да') 
                          ].shape[0]
    

    #  ------------------------------------------Взрослые Выезды ВРАЧЕЙ -------заведующе
    df_vzrosl_doctor = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        # &  (df['Вид_визита'].str.contains("врач") == True)   
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )      
                          ].shape[0] 

    #  ------------------------------------------Взрослые Выезды среднего м/п ------- медицинской 
    df_vzrosl_nurse = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))        
                          ].shape[0]   
    
     #  ------------------------------------------------------------------------Дети ВСЕГО -------
    df_deti_all = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~(df['Дети'].isna()))
                          ].shape[0]
    
    
    #  ------------------------------------------Дети  Выезды ВРАЧЕЙ -------
    df_deti_doctor = df.loc[ 
                          ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )     
                          ].shape[0]  
    
    #  ------------------------------------------Дети  Выезды среднего м/п -------
    df_deti_nurse = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        # &  (df['Оборуд'] != '#Н/Д') | (df['Набл_РЦ'] != 'Нет')
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))            
                          ].shape[0]  
    
    

    # return df
    return df_vzrosl_all, df_vzrosl_doctor, df_vzrosl_nurse, df_deti_all, df_deti_doctor, df_deti_nurse

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция формирования Датафрейма данных по выездам к РЕСПИРАТОРНЫМ за период  <br></b>            </div>

In [ ]:
# ------------------------------------------------------------------------------------ по признаку "1"--------------------------
def f_create_df_vizit_vps_k_resper(file_name, start, finish):
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла VPS ----------------------

    df = load_data_vps(file_name,finish)
    df = df.loc[ (df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish )]
    
    #  ------------------------------------------Взрослые ВСЕГО -------
    df_vzrosl_all_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        # &  (df['Клиент.БК'] != 'Да') 
        
                          ].shape[0]


    #  ------------------------------------------Взрослые Выезды ВРАЧЕЙ -------заведующе
    df_vzrosl_doctor_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )      
                          ].shape[0] 

    #  ------------------------------------------Взрослые Выезды среднего м/п ------- медицинской 
    df_vzrosl_nurse_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))        
                          ].shape[0]   
    
     #  ------------------------------------------------------------------------Дети ВСЕГО -------
    df_deti_all_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                         &  (~(df['Дети'].isna()))
                         &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                          ].shape[0]
    
    
    #  ------------------------------------------Дети  Выезды ВРАЧЕЙ -------
    df_deti_doctor_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("врач") == True) | (df['Вид_визита'].str.contains("заведующе") == True) )     
                          ].shape[0]  
    
    #  ------------------------------------------Дети  Выезды среднего м/п -------
    df_deti_nurse_resper = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (~df['Дети'].isna())  
                        &  ((df['Выдано_респ_оборуд'] != 0) | (df['Набл_РЦ'] != 'Нет'))
                        &  ( (df['Вид_визита'].str.contains("сестр") == True) | (df['Вид_визита'].str.contains("брат") == True) | (df['Вид_визита'].str.contains("Выдача") == True))            
                          ].shape[0]  
    


    # return df
    df_vzrosl_all_resper_df = df.loc[ 
                           ((df['Дата_визита'] >= start)   &   (df['Дата_визита'] <  finish ))
                        &  (df['Дети'].isna())
                        &  ((df['Выдано_респ_оборуд'] != '0') | (df['Набл_РЦ'] != 'Нет'))
                          ]
    return df_vzrosl_all_resper, df_vzrosl_doctor_resper, df_vzrosl_nurse_resper, df_deti_all_resper, df_deti_doctor_resper, df_deti_nurse_resper, df_vzrosl_all_resper_df

#
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info">
<div class="alert alert-block alert-warning">
<div class="alert alert-block alert-success">
<div class="alert alert-block alert-danger">
     -->
<font color= green size=10>
<b> ВЫВОД РЕЗУЛЬТАТОВ по выездам <br>
</b>            </div>

In [ ]:
start = '2023-01-01'
finish = '2023-07-01'
# file_name = 'впс_1й_кв_0704.csv'
# file_name = 'впс_1й_кв_0704_выданое_респ.csv'
file_name = 'впс_1е_полугод_с_выданным_респ.csv'

df_vps = f_create_df_vizit_vps(file_name, start, finish)
df_resp = f_create_df_vizit_vps_k_resper(file_name, start, finish)

a = f'Совершено выездов ВПС в период {start} - {finish}:'
display_markdown(f'''# {a}
| Параметр | Всего | На дому | Из них к пациентам с респират.поддеркой |
|---:|:-|:-|:-|
| Совершено выездов к взрослым            | __{df_vps[0]}__ |__{df_vps[0]}__ |__{df_resp[0]}__ |
| Совершено выездов врачами  (к взрослым) | __{df_vps[1]}__ |__{df_vps[1]}__ |__{df_resp[1]}__ |
| Совершено выездов сред.м/п (к взрослым) | __{df_vps[2]}__ |__{df_vps[3]}__ |__{df_resp[2]}__ |
| Совершено выездов к детям               | __{df_vps[3]}__ |__{df_vps[3]}__ |__{df_resp[3]}__ |
| Совершено выездов врачами  (к детям)    | __{df_vps[4]}__ |__{df_vps[4]}__ |__{df_resp[4]}__ |
| Совершено выездов сред.м/п (к детям)    | __{df_vps[5]}__ |__{df_vps[5]}__ |__{df_resp[5]}__ |

''', raw=True)

In [ ]:
start = '2023-01-01'
finish = '2023-07-01'
# file_name = 'впс_1й_кв_0704.csv'
# file_name = 'впс_1й_кв_0704_выданое_респ.csv'
file_name = 'впс_1е_полугод_с_выданным_респ_без_аспират.csv'

df_vps = f_create_df_vizit_vps(file_name, start, finish)
df_resp = f_create_df_vizit_vps_k_resper(file_name, start, finish)

a = f'Совершено выездов ВПС в период {start} - {finish}:'
display_markdown(f'''# {a}
| Параметр | Всего | На дому | Из них к пациентам с респират.поддеркой(АСПИРАТОРЫ не РЕСПИРАТОРКА) |
|---:|:-|:-|:-|
| Совершено выездов к взрослым            | __{df_vps[0]}__ |__{df_vps[0]}__ |__{df_resp[0]}__ |
| Совершено выездов врачами  (к взрослым) | __{df_vps[1]}__ |__{df_vps[1]}__ |__{df_resp[1]}__ |
| Совершено выездов сред.м/п (к взрослым) | __{df_vps[2]}__ |__{df_vps[3]}__ |__{df_resp[2]}__ |
| Совершено выездов к детям               | __{df_vps[3]}__ |__{df_vps[3]}__ |__{df_resp[3]}__ |
| Совершено выездов врачами  (к детям)    | __{df_vps[4]}__ |__{df_vps[4]}__ |__{df_resp[4]}__ |
| Совершено выездов сред.м/п (к детям)    | __{df_vps[5]}__ |__{df_vps[5]}__ |__{df_resp[5]}__ |

''', raw=True)

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Расчет сведений по ДИАГНОЗАМ из 605го <br>
</b>            </div>

###
<div class="alert alert-block alert-info">

<!-- <div class="alert alert-block alert-info">
<div class="alert alert-block alert-warning">
<div class="alert alert-block alert-success">
<div class="alert alert-block alert-danger">
     -->
<font size=3>
<b>Распишем ДИАГНОЗЫ (внесем новые колонки-принаки в датафрейм)</b>               

In [ ]:
def diagnoz(df_podgotovlen):
    
         #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла 605го ----------------------

    df_concat = df_podgotovlen
    # ----- к нижнему регистру, все, что  в Основной диагноз
    df_concat['Диагноз'] = df_concat['Диагноз'].str.lower()

    # ---- новый Признак ДИАГНОЗА проставлен-----
    df_concat["признак_Диагноза"] = np.nan

    # ---------------------------------------------------------------- новый признак Онкология-----
    df_concat["Onko"] = np.nan

    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("рак") == True)           
              |(df_concat['Диагноз'].str.contains("зно") == True)           
              |(df_concat['Диагноз'].str.contains("образовани") == True)           
              |(df_concat['Диагноз'].str.contains("онкол") == True)           
              |(df_concat['Диагноз'].str.contains("миелом") == True)           
              |(df_concat['Диагноз'].str.contains("опухо") == True)    
              |(df_concat['Диагноз'].str.contains("мтс") == True)  
              |(df_concat['Диагноз'].str.contains("пммр") == True)  
              |(df_concat['Диагноз'].str.contains("злокач") == True)  
              |(df_concat['Диагноз'].str.contains("mts") == True)  
              |(df_concat['Диагноз'].str.contains("злокач") == True)  
              |(df_concat['Диагноз'].str.contains("злокач") == True)  
              |(df_concat['Диагноз'].str.contains("сарко") == True)  


              |(df_concat['Диагноз'].str.contains("лейкоз") == True)  
              |(df_concat['Диагноз'].str.contains("c00") == True)  
              |(df_concat['Диагноз'].str.contains("c97") == True)  
              |(df_concat['Диагноз'].str.contains("d48") == True) 


                  |(df_concat['Диагноз'].str.contains("с00") == True)  
              |(df_concat['Диагноз'].str.contains("с97") == True)  
              |(df_concat['Диагноз'].str.contains("d48") == True) 

              |(df_concat['Диагноз'].str.contains("c43") == True) 
              |(df_concat['Диагноз'].str.contains("d32") == True) 
              |(df_concat['Диагноз'].str.contains("c16") == True) 
              |(df_concat['Диагноз'].str.contains("c34") == True) 

         |(df_concat['Диагноз'].str.contains("с80") == True) 
         |(df_concat['Диагноз'].str.contains("с18") == True) 
         |(df_concat['Диагноз'].str.contains("глиобластома головного мозга") == True)  

                    # |(df_gosp['Диагноз'].str.contains("d48") == True)  
                    # |(df_gosp['Диагноз'].str.contains("d48") == True)  
    , 'Onko' ] = 'Onko'

    df_concat.loc[~df_concat['Onko'].isna(), 'признак_Диагноза'] = 'Есть'





    # --------------------------------------------------------------------- новый признак Неврология-----
    df_concat["Неврология"] = np.nan

    # ---- --------------------------- признак Неврология-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("неврология") == True)           
              |(df_concat['Диагноз'].str.contains("g00 - g99") == True)           
              |(df_concat['Диагноз'].str.contains("g00-g98") == True)           
              |(df_concat['Диагноз'].str.contains("g00") == True)           
              |(df_concat['Диагноз'].str.contains("болезни нервной системы") == True)           


    #                 |(df_gosp['Диагноз'].str.contains("d48") == True)  
    #                 |(df_gosp['Диагноз'].str.contains("d48") == True)  
    #                 |(df_gosp['Диагноз'].str.contains("d48") == True)  
    , 'Неврология' ] = 'Неврология'

    df_concat.loc[~df_concat['Неврология'].isna(), 'признак_Диагноза'] = 'Есть'

    # ---------------------------------------------------------------------- новый признак Терапия-----
    df_concat["Терапия"] = np.nan

    # ---------------------------------------- новый признак Терапия-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains(" болезни системы кровообращения") == True)  
               | (df_concat['Диагноз'].str.contains("i00-i99") == True)

               | (df_concat['Диагноз'].str.contains("k00-k93") == True)
               | (df_concat['Диагноз'].str.contains("болезни органов пищеварения") == True)

               | (df_concat['Диагноз'].str.contains("j00-j99") == True)
               | (df_concat['Диагноз'].str.contains("болезни органов дыхания") == True)

               | (df_concat['Диагноз'].str.contains("m00-m99") == True)
               | (df_concat['Диагноз'].str.contains("костно-мышечной") == True)

               | (df_concat['Диагноз'].str.contains("n00-n99") == True)
               | (df_concat['Диагноз'].str.contains("мочеполовой системы") == True)

               | (df_concat['Диагноз'].str.contains("d82") == True)

               | (df_concat['Диагноз'].str.contains("d50-d89") == True)
               | (df_concat['Диагноз'].str.contains("болезни крови, кроветворных") == True)

               | (df_concat['Диагноз'].str.contains("цвб хигм") == True)

               | (df_concat['Диагноз'].str.contains("м00-м050") == True)

    #            | (df_concat['Диагноз'].str.contains("м00-м050") == True)
    #            | (df_concat['Диагноз'].str.contains("болезни органов дыхания") == True)




    , 'Терапия' ] = 'Терапия'

    df_concat.loc[~df_concat['Терапия'].isna(), 'признак_Диагноза'] = 'Есть'


    # ---------------------------------------------------------------------------- новый признак Педиатрия-----
    df_concat["Педиатрия"] = np.nan

    # --------------------------------------- новый признак Педиатрия-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("p00-p96") == True)  
               | (df_concat['Диагноз'].str.contains("отдельные состояния, возникающие") == True)
               | (df_concat['Диагноз'].str.contains("р00-р96") == True)
               # | (df_concat['Диагноз'].str.contains("s00-t98") == True)
               # | (df_concat['Диагноз'].str.contains("травмы, отравления и некоторые") == True)

        , 'Педиатрия' ] = 'Педиатрия'

    df_concat.loc[~df_concat['Педиатрия'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак Врожденные_аномалии-----  
    df_concat["Врожденные"] = np.nan

    # --------------------------------------- новый признак Врожденные_аномалии-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("q00-q99") == True)  
               | (df_concat['Диагноз'].str.contains("пороки развития") == True)

               # | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               # | (df_concat['Диагноз'].str.contains("s00-t98") == True)

        , 'Врожденные' ] = 'Врожденные'

    df_concat.loc[~df_concat['Врожденные'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак ВИЧ----- 
    df_concat["ВИЧ"] = np.nan

    # --------------------------------------- новый признак ВИЧ-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("в20.7") == True)  
               | (df_concat['Диагноз'].str.contains("вич-инфекция") == True)

               # | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               # | (df_concat['Диагноз'].str.contains("s00-t98") == True)

        , 'ВИЧ' ] = 'ВИЧ'

    df_concat.loc[~df_concat['ВИЧ'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак Психиатрия-----
    df_concat["Психиатрия"] = np.nan


    # --------------------------------------- новый признак Психиатрия-----  
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("f00-f99") == True)  
               | (df_concat['Диагноз'].str.contains("психические расстройства") == True)

               | (df_concat['Диагноз'].str.contains("f.20") == True)
               | (df_concat['Диагноз'].str.contains("f.02") == True)


               | (df_concat['Диагноз'].str.contains("шизофрения") == True)

               # | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               # | (df_concat['Диагноз'].str.contains("f.20.004) шизофрения") == True)

        , 'Психиатрия' ] = 'Психиатрия'

    df_concat.loc[~df_concat['Психиатрия'].isna(), 'признак_Диагноза'] = 'Есть'


    # ----------------------------------------- новый признак Прочие-----   
    df_concat["Прочие"] = np.nan

    # --------------------------------------- новый признак Прочие-----
    df_concat.loc[
                 (df_concat['Диагноз'].str.contains("инфекционные и паразитарные болезни") == True)  
               | (df_concat['Диагноз'].str.contains("a00-b99") == True)
               | (df_concat['Диагноз'].str.contains("диагнозтест") == True)
               | (df_concat['Диагноз'].str.contains("s00-t98") == True)
               | (df_concat['Диагноз'].str.contains("травмы, отравления и некоторые") == True)
               | (df_concat['Диагноз'].str.contains("короновирусная инфекция") == True)
               | (df_concat['Диагноз'].str.contains("r00-r99") == True)
               | (df_concat['Диагноз'].str.contains("l00-l99") == True)
               | (df_concat['Диагноз'].str.contains("болезни кожи и подкожной клетчатки") == True)
               | (df_concat['Диагноз'].str.contains("h00-h59") == True)
               | (df_concat['Диагноз'].str.contains("e00-e90") == True)
               | (df_concat['Диагноз'].str.contains("эндокринной системы") == True)
               | (df_concat['Диагноз'].str.contains("алкогольный цирроз") == True)


              | (df_concat['Диагноз'].str.contains("е00-е90") == True)


    #            | (df_concat['Диагноз'].str.contains("е00-е90") == True)
    #            | (df_concat['Диагноз'].str.contains("костно-мышечной") == True)

        , 'Прочие' ] = 'Прочие'

    df_concat.loc[~df_concat['Прочие'].isna(), 'признак_Диагноза'] = 'Есть'

    # ------------------------ после всех ПРИЗНАКОВ остатки - в ПРОЧИЕ ----
    df_concat.loc[df_concat['признак_Диагноза'].isna(), 'Прочие'] = 'Прочие'

    df_concat.loc[~df_concat['Прочие'].isna(), 'признак_Диагноза'] = 'Есть'

    return df_concat

###
<div class="alert alert-block alert-warning">

<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> <font size=3>
<b> функция подсчета пациентов по ВИДУ диагноза  </b>               

#### ----------------------- функция ДИАГНОЗА --------------

In [ ]:
# ----------------------- функция расчета Неврология--------------
def calculate_count_diagnoz(df, start, finish, diagnoz):
    #     ---------------------------------------------------------------- ВЗРОСЛЫХ ----------------------------------------------------------
    diagnoz = diagnoz
    vzros = df.loc[ 
                            (df['Дата_первых_сведений'] < finish)                                                      # дата внесения в реестр - ранее date_start
                          & ( (df['Дата_выбытия'].isnull())   | (df['Дата_выбытия'] >= start) )                   # дата выбытия - еще не выбыл или дата выбытия - после date_start
                          & (  df[diagnoz] == diagnoz)                                                                  # ОНКО = ДА
                          & ( (df['Дети']).isna())                                                                         # Дети = НЕТ 
                        ].shape[0]
    
    
    vzros_trud = df.loc[ 
                            (df['Дата_первых_сведений'] < finish)                                                      # дата внесения в реестр - ранее date_start
                          & ( (df['Дата_выбытия'].isnull())   | (df['Дата_выбытия'] >= start) )                   # дата выбытия - еще не выбыл или дата выбытия - после date_start
                          & (  df[diagnoz] == diagnoz)                                                                  # ОНКО = ДА
                          & ( (df['Дети']).isna())                                                                         # Дети = НЕТ 
                          & ( ~(df['Категория']).isna())                                                                         # Дети = НЕТ 
                        ].shape[0]
    
    
    
    
    
    
    
    #     ---------------------------------------------------------------- Дети ----------------------------------------------------------
    deti = df.loc[ 
                            (df['Дата_первых_сведений'] < finish)                                                      # дата внесения в реестр - ранее date_start
                          & ( (df['Дата_выбытия'].isnull())   | (df['Дата_выбытия'] >= start) )                   # дата выбытия - еще не выбыл или дата выбытия - после date_start
                         
                          & (  df[diagnoz] == diagnoz)                                                                 # ОНКО = No
                          & ( ~(df['Дети']).isna())                                                                         # Дети = НЕТ 
                        ].shape[0]    
    
    return vzros, vzros_trud, deti    

In [ ]:
# --------------------------------------------------------------------------------функция подсчета ВСЕГО пациентов --------------
#--------------------------------------------  Количество пациентов, получателей помощи (чел.)
def def_data_all_clients(df, start, finish):

    # data_all_clients = df.loc[ (df['Дата_вкл_реестр'] <= finish)  
    #                          &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )].shape[0] 

    
    data_all_clients_vzros = df.loc[ 
                                 (df['Дата_первых_сведений'] < finish)  &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )
                                  & ( (df['Дети'].isnull()))   
        
                       ].shape[0] 
    
    data_all_clients_vzros_trud = df.loc[ 
                                 (df['Дата_первых_сведений'] < finish)  &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )
                                  &  df['Дети'].isnull()
                                  & ( ~(df['Категория'].isnull()))
                                      ].shape[0] 
        
        
    
    data_all_clients_deti = df.loc[ 
                                 (df['Дата_первых_сведений'] < finish)  &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )
                                 & ( ~(df['Дети'].isnull()))   
        
                       ].shape[0] 
    
    return  data_all_clients_vzros, data_all_clients_vzros_trud, data_all_clients_deti

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Результат подсчета ДИАГНОЗОВ в периоде <br></b>            </div>

In [ ]:
def poluchateli_v_reestre(name_605, name_preparat, start, finish, bmchu):
    
    # ---------------- формируем df по чистому 605-му ----
    df_cpp = load_data_605(name_605, bmchu, finish)
    # ---------------- формируем df по НС ----
    df_preparat = load_preparat(name_preparat, finish)    
        
    # ----------- функция подсчета Количество пациентов, получателей помощи (чел.) --
    df_605_period = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[0]
    df_605_period_trud = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[2]
    
#     # ----------- Оставим только нужные поля 605го для соединения с данными СДВ -------------
    df_605_period = df_605_period[['Пациент', 'ДР', 'Ключ', 'Категория']]

    df_mer = df_605_period.merge(df_preparat[['Ключ','preparat', 'Возраст', 'Дети']], how='left', on='Ключ')
# 
#     # ----------- Взрослые preparat (уникальные) -------------
    df_preparat_vzros = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] > 17)].drop_duplicates()
#     # ----------- Взрослые preparat СТАРШЕ ТРУД (уникальные) -------------
    df_preparat_vzros_trud = df_preparat_vzros.loc[~df_preparat_vzros.Категория.isna()].drop_duplicates()
#     # ----------- Взрослые Дети (уникальные) -------------
    df_preparat_deti = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] <= 17)].drop_duplicates()


    return df_605_period.shape[0], df_605_period_trud, df_preparat_vzros.shape[0], df_preparat_vzros_trud.shape[0], df_preparat_deti.shape[0]
    

####
<div class="alert alert-block alert-warning">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция Получатели (ДЛЯ ДАТАФРЕЙМОВ) <br></b>            </div>

In [ ]:
def poluchateli_v_reestre_df(name_605, name_preparat, start, finish, bmchu):
    
    # ---------------- формируем df по чистому 605-му ----
    df_cpp = load_data_605(name_605, bmchu, finish)
    # ---------------- формируем df по НС ----
    df_preparat = load_preparat(name_preparat, finish)    
        
    # ----------- функция подсчета Количество пациентов, получателей помощи (чел.) --
    df_605_period = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[0]
    df_605_period_trud = def_data_all_clients_data_vkl_reestr(df_cpp, start, finish)[2]
    
#     # ----------- Оставим только нужные поля 605го для соединения с данными СДВ -------------
    df_605_period = df_605_period[['Пациент', 'ДР', 'Ключ', 'Категория']]

    df_mer = df_605_period.merge(df_preparat[['Ключ','preparat', 'Возраст', 'Дети']], how='left', on='Ключ')
# 
#     # ----------- Взрослые preparat (уникальные) -------------
    df_preparat_vzros = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] > 17)].drop_duplicates()
#     # ----------- Взрослые preparat СТАРШЕ ТРУД (уникальные) -------------
    df_preparat_vzros_trud = df_preparat_vzros.loc[~df_preparat_vzros.Категория.isna()].drop_duplicates()
#     # ----------- Взрослые Дети (уникальные) -------------
    df_preparat_deti = df_mer.loc[(~df_mer.preparat.isna()) & (df_mer['Возраст'] <= 17)].drop_duplicates()


    return df_605_period, df_605_period_trud, df_preparat_vzros, df_preparat_vzros_trud, df_preparat_deti, df_preparat

<hr>

<font color= green size=10>__ВЫВОД РЕЗУЛЬТАТОВ__
<hr>


In [ ]:
# ---------------------------- ИТОГИ ---------------
start = '2023-01-01'
finish = '2023-07-01'

# file_name = '605_рц_бк_1204.csv'
file_name = '605_рц_бк_0407.csv'

bmchu = 'не учитываем'
# bmchu = 'учитываем'
bk = 'с учетом БК'
# bk = 'без учета БК'


# ---------------------------- Готовим датафрейм РЕЕСТРА ---------------
df_reestr = load_data_605(file_name, bmchu, bk,finish)

# ---------------------------- Готовим датафрейм c РАСПИСАНЫМИ ДИАГНОЗАМИ ---------------
df_insert_diagnoz = diagnoz(df_reestr)

s_list = ['Onko', 'Неврология', 'Терапия',  'Педиатрия', 'Врожденные', 'ВИЧ', 'Психиатрия',  'Прочие']

#  ------------------- считаем в цикле пациентов для  ['Неврология', 'Терапия',  'Педиатрия', 'Врожденные', 'ВИЧ', 'Психиатрия',  'Прочие'] --
list_diagn_pac = []
for diagn in s_list:
    list_diagn_pac.append(calculate_count_diagnoz(df_insert_diagnoz, start, finish, diagn))
    

    
data_stat_all = def_data_all_clients(df_insert_diagnoz, start, finish)
    
god_start = start.split('-')[0]
mes_start = start.split('-')[1]
day_start = start.split('-')[2]

god_finish = finish.split('-')[0]
mes_finish = finish.split('-')[1]
day_finish = finish.split('-')[2]

a = f'Статистика по ДИАГНОЗАМ в период с {day_start}.{mes_start}.{god_start} по {day_finish}.{mes_finish}.{god_finish}:'

display_markdown(f'''
# {a}
| Параметр | Взрослые | В т.ч. трудоспособ | Дети |
|---:|:-|:-|:-|
| __Всего пациентов__{bk}__        | {data_stat_all[0]} | {data_stat_all[1]} |{data_stat_all[2]} |
| пациенты с диагнозом __"Онко"__  | {list_diagn_pac[0][0]} | {list_diagn_pac[0][1]} |{list_diagn_pac[0][2]} |
| пациенты с диагнозом __"Неврология"__  | {list_diagn_pac[1][0]} | {list_diagn_pac[1][1]} |{list_diagn_pac[1][2]} |
| пациенты с диагнозом __"Терапия"__  | {list_diagn_pac[2][0]} | {list_diagn_pac[2][1]} |{list_diagn_pac[2][2]} |
| пациенты с диагнозом __"Педиатрия"__  | {list_diagn_pac[3][0]} | {list_diagn_pac[3][1]} |{list_diagn_pac[3][2]} |
| пациенты с диагнозом __"Врожденные"__  | {list_diagn_pac[4][0]} | {list_diagn_pac[4][1]} |{list_diagn_pac[4][2]} |
| пациенты с диагнозом __"ВИЧ"__  | {list_diagn_pac[5][0]} | {list_diagn_pac[5][1]} |{list_diagn_pac[5][2]} |
| пациенты с диагнозом __"Психиатрия"__  | {list_diagn_pac[6][0]} | {list_diagn_pac[6][1]} |{list_diagn_pac[6][2]} |
| пациенты с диагнозом __"Прочие"__  | {list_diagn_pac[7][0]} | {list_diagn_pac[7][1]} |{list_diagn_pac[7][2]} |
| __Статус по БМЧУ__  | __{bmchu}__ | --|-- |-- |




''', raw=True)

In [ ]:
data_stat_all[4]

In [ ]:
data_stat_all[4].to_excel('data_605_2023_бмчу.xlsx')